In [60]:
import requests, pandas
import numpy as np
from IPython.display import Image,HTML
pandas.set_option('display.max_colwidth', -1)

In [65]:

def reindex_from_1(df):
    df2 = df.reset_index(drop=True)
    df2.index = df2.index+1
    return df2

def path_to_image_html(path):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''

    return '<img src="'+ path + '" style=max-height:20px;"/>'

In [164]:
game_df = pandas.read_csv("data/fbs_games_short.csv")
team_df = pandas.read_csv("data/fbs_teams_short.csv")

In [165]:
HTML(fbs_team_df.to_html(escape=False ,formatters=dict(logo=path_to_image_html)))

,logo,school,mascot,conference,id
1,,Air Force,Falcons,Mountain West,2005
2,,Akron,Zips,Mid-American,2006
3,,Alabama,Crimson Tide,SEC,333
4,,Appalachian State,Mountaineers,Sun Belt,2026
5,,Arizona,Wildcats,Pac-12,12
6,,Arizona State,Sun Devils,Pac-12,9
7,,Arkansas,Razorbacks,SEC,8
8,,Arkansas State,Red Wolves,Sun Belt,2032
9,,Army,Black Knights,FBS Independents,349
10,,Auburn,Tigers,SEC,2


In [176]:
wins_dict = {}
losses_dict = {}
logo_dict = {}
score_dict = {}
for index,team_row in team_df.iterrows():
    team = team_row['school']
    wins_dict[team]=[]
    losses_dict[team]=[]
    logo_dict[team]=team_row['logo']
    score_dict[team] = 0
#num_wins = [len(wins_dict[x]) for x in wins_dict.keys()]

In [177]:
#build w-l relationships
for index,game in game_df.iterrows():
    if game['home_points']>game['away_points']:
        winning_team = game['home_team']
        losing_team = game['away_team']   
    else:
        winning_team = game['away_team']
        losing_team = game['home_team']
    wins_dict[winning_team].append(losing_team)
    losses_dict[losing_team].append(winning_team)    

In [178]:
for team in score_dict:
    print(team)
    for team_defeated in wins_dict[team]:
        score_dict[team] += len(wins_dict[team_defeated])
        print("+",team_defeated,len(wins_dict[team_defeated]))
    for team_lost_to in losses_dict[team]:
        score_dict[team] -= len(losses_dict[team_lost_to])
        print("-",team_lost_to,len(losses_dict[team_lost_to]))
    print("*",score_dict[team])

Ohio State
+ Florida Atlantic 4
+ Cincinnati 6
+ Indiana 5
+ Miami (OH) 3
+ Nebraska 4
+ Michigan State 4
+ Northwestern 1
+ Wisconsin 6
* 33
LSU
+ Georgia Southern 3
+ Texas 5
+ Vanderbilt 2
+ Utah State 3
+ Florida 5
+ Mississippi State 3
+ Auburn 6
* 27
Penn State
+ Buffalo 3
+ Pittsburgh 4
+ Maryland 2
+ Purdue 2
+ Iowa 6
+ Michigan 6
+ Michigan State 4
* 27
Clemson
+ Georgia Tech 2
+ Texas A&M 4
+ Syracuse 2
+ Charlotte 2
+ North Carolina 4
+ Florida State 4
+ Louisville 4
+ Boston College 3
* 25
SMU
+ Arkansas State 3
+ North Texas 2
+ Texas State 1
+ TCU 3
+ South Florida 3
+ Tulsa 2
+ Temple 4
+ Houston 2
* 20
Cincinnati
+ UCLA 3
+ Miami (OH) 3
+ Marshall 4
+ UCF 5
+ Houston 2
+ Tulsa 2
- Ohio State 0
* 19
Michigan
+ Middle Tennessee 2
+ Army 2
+ Rutgers 2
+ Iowa 6
+ Illinois 4
+ Notre Dame 5
- Wisconsin 2
- Penn State 0
* 19
Minnesota
+ Fresno State 2
+ Georgia Southern 3
+ Purdue 2
+ Illinois 4
+ Nebraska 4
+ Rutgers 2
+ Maryland 2
* 19
Auburn
+ Oregon 6
+ Tulane 4
+ Kent Sta

- Utah 1
* -5
Washington State
+ New Mexico State 0
+ Houston 2
+ Colorado 3
- UCLA 5
- Utah 1
- Arizona State 3
- Oregon 1
* -5
Arizona
+ Texas Tech 2
+ UCLA 3
+ Colorado 3
- Hawai'i 3
- Washington 3
- USC 3
- Stanford 4
* -5
Georgia Tech
+ South Florida 3
+ Miami 3
- Clemson 0
- Temple 3
- North Carolina 4
- Duke 4
* -5
Ohio
+ Buffalo 3
+ Kent State 2
+ Ball State 3
- Pittsburgh 3
- Marshall 3
- Louisiana 2
- Northern Illinois 5
* -5
West Virginia
+ NC State 3
+ Kansas 2
- Missouri 3
- Texas 3
- Iowa State 3
- Oklahoma 1
* -5
Eastern Michigan
+ Coastal Carolina 2
+ Illinois 4
+ Western Michigan 4
- Kentucky 4
- Central Michigan 4
- Ball State 4
- Toledo 3
* -5
Coastal Carolina
+ Kansas 2
+ UMass 1
- Eastern Michigan 4
- Appalachian State 0
- Georgia State 2
- Georgia Southern 3
* -6
Charlotte
+ UMass 1
+ North Texas 2
- Appalachian State 0
- Clemson 0
- Florida Atlantic 3
- Florida International 4
- Western Kentucky 2
* -6
NC State
+ East Carolina 1
+ Ball State 3
+ Syracuse 2
- West

In [179]:
record_dict = {}
for team in wins_dict:
    num_wins = len(wins_dict[team])
    num_losses = len(losses_dict[team])
    record_dict[team] = str(num_wins) + "-" +  str(num_losses)

In [180]:
team_df['score'] = team_df['school'].map(score_dict)
team_df['record'] = team_df['school'].map(record_dict)
team_df = team_df.sort_values(by='score', ascending=False)
team_df = reindex_from_1(team_df)
team_df = team_df[['logo','school','mascot','conference','record','score']]
HTML(team_df.to_html(escape=False ,formatters=dict(logo=path_to_image_html)))

,logo,school,mascot,conference,record,score
1,,Ohio State,Buckeyes,Big Ten,8-0,33
2,,Penn State,Nittany Lions,Big Ten,7-0,27
3,,LSU,Tigers,SEC,7-0,27
4,,Clemson,Tigers,ACC,8-0,25
5,,SMU,Mustangs,American Athletic,8-0,20
6,,Cincinnati,Bearcats,American Athletic,6-1,19
7,,Michigan,Wolverines,Big Ten,6-2,19
8,,Minnesota,Golden Gophers,Big Ten,7-0,19
9,,Auburn,Tigers,SEC,6-2,19
10,,Oregon,Ducks,Pac-12,6-1,18
